In [18]:
# import libraries
import json
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer

#### Step 1: Prepare documents

In [6]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [7]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

documents[1]

{'text': 'GitHub - DataTalksClub/data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp'}

#### Step 2: Create Embeddings using Pretrained Models

In [9]:
model = SentenceTransformer("all-mpnet-base-v2")

c:\Users\tellw\anaconda3\envs\llm\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\tellw\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [10]:
len(model.encode("This is a simple sentence"))

768

In [11]:
documents[1]

{'text': 'GitHub - DataTalksClub/data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp'}

In [12]:
# created the dense vector using the pre-trained model
operations = []

for doc in documents:
    # transforming the title into an embedding using the model
    doc["text_vector"] = model.encode(doc["text"]).tolist()
    operations.append(doc)

#### Step 3: Setup ElasticSearch connection

In [19]:
es_client = Elasticsearch('http://localhost:9200') 
es_client.info()

ObjectApiResponse({'name': 'ca85f9145c0f', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'OeNw2QEoT_ubbord9TCjpw', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

#### Step 4: Create Mappings and Index

In [20]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "text_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
        }
    }
}

In [21]:
index_name = "course-questions"

es_client.indices.delete(index = index_name, ignore_unavailable = True)
es_client.indices.create(index = index_name, body = index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

#### Step 5: Add documents into index

In [22]:
for doc in operations:
    try:
        es_client.index(index = index_name, document = doc)
    except Exception as e:
        print(e)

#### Step 6: Create end user query

In [23]:
search_term = "windows or mac?"
vector_search_term = model.encode(search_term)

In [24]:
query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000, 
}

In [25]:
res = es_client.search(
    index = index_name, 
    knn = query,
    source = ["text", "section", "question", "course"])

res["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'y9wv1ZAB8jFkQwOzLH8o',
  '_score': 0.7147919,
  '_source': {'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully'}},
 {'_index': 'course-questions',
  '_id': '7dwv1ZAB8jFkQwOz5oJ2',
  '_score': 0.6107274,
  '_source': {'question': 'WSL instructions',
   'course': 'mlops-zoomcamp',
   'section': 'Module 1: Introduction',
   'text': 'If you wish to use WSL on your windows machine, here are the setup instructions:\nCommand: Sudo apt install wget\nGet Anaconda download address here. wget <download address>\nTurn on Docker Desktop WFree Download | AnacondaSL2\nCommand: git clone <github repository address>\nVSCODE on WSL\nJupyter: pip3 install jupyter\nAdded by Gregory Morris (gwm1980@gmail.com)\nAll in all softwares at

#### Step 7: Perform Keyword search with Semantic Search (Hybrid/ Advanced Search)

In [27]:
knn_query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000
}

In [28]:
response = es_client.search(
    index = index_name,
    query = {
        "match": {"section": "General course-related questions"},
    },
    knn = knn_query,
    size = 5
)
response["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'y9wv1ZAB8jFkQwOzLH8o',
  '_score': 11.598243,
  '_source': {'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully',
   'section': 'General course-related questions',
   'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'text_vector': [-0.026965470984578133,
    -0.0006261608214117587,
    -0.016629496589303017,
    0.05285150185227394,
    0.0547652430832386,
    -0.03133988752961159,
    0.029942601919174194,
    -0.048085641115903854,
    0.04467552900314331,
    0.005839458201080561,
    0.016233086585998535,
    0.012001135386526585,
    -0.031222272664308548,
    0.016600510105490685,
    -0.048869043588638306,
    -0.06496305018663406,
    0.04643421620130539,
    -0.00929773598909378,
    -0.0642528384923935,
    -0.013732712715864182,
    -0.015976224094629288,
    0.008629555813968182,
    -0.024478992447